In [1]:
import os
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = ".20"

In [2]:
import pandas as pd
from model import Brain
from submodels import factories
import matplotlib.pyplot as plt
import pandas as pd
from jf.db import DB
from lib.plot import plot_2d_positions_colors
from lib.sde.grn2 import GRNMain2
from lib.sde.grn import GRNMain, grn_from_json
from lib.sde.cell2 import Cell2, CellBatch
from lib.sde.mutate import mutate_grn2
from jf.profiler import Profiler

In [31]:
from itertools import accumulate
import numpy as np
from collections import defaultdict, Counter
import jax.numpy as jnp
import numpy as np

In [4]:
HISTORY = defaultdict(dict)

In [5]:
END_TIME = 54

In [6]:
HALL_OF_FAME = []

In [7]:
REF = pd.read_csv("output/results/setup_basic/export/ref_basic.csv")  # ref is a mean

In [8]:
def individual_generator():
    return Solution(GRNMain2(5, 0, 5))

In [9]:
# set a similar grn
ref_grn = """
{"object_name": "GRNMain2", "nb_start_gene": 5, "nb_off_trees": 0, "nb_genes": 5,
"params": [[2.007115364074707, 0.3403625786304474, 1.3039988279342651, 2.0225210189819336, 1.5461641550064087],
[2.375922203063965, 7.7354350090026855, 5.84783411026001, 6.466616153717041, 8.159892082214355],
[7.479958534240723, 5.624154090881348, 8.094839096069336, 1.19766366481781, 8.925475120544434],
[9.56630802154541, 3.818467378616333, 9.031471252441406, 7.2058916091918945, 1.283474326133728],
[6.358775615692139, 5.765143394470215, 9.488754272460938, 2.5621650218963623, 5.002293586730957],
[3.9804110527038574, 4.080349922180176, 9.770094871520996, 9.090896606445312, 4.010365009307861],
[5.9285149574279785, 3.148050546646118, 4.948235511779785, 3.5156848430633545, 4.3454718589782715],
[8.12299919128418, 2.998713493347168, 5.796504020690918, 9.65429973602295, 0.45337215065956116]],
"trees": [{"op": "Var", "val": 2}, {"op": "Var", "val": 0},
{"op": "And", "F1": {"op": "Or", "F1": {"op": "Var", "val": 1},
"F2": {"op": "Or", "F1": {"op": "Var", "val": 2}, "F2": {"op": "Var", "val": 3}}},
"F2": {"op": "Not", "F": {"op": "Or", "F1": {"op": "Var", "val": 4}, "F2": {"op": "Var", "val": 0}}}},
{"op": "And", "F1": {"op": "Var", "val": 4}, "F2": {"op": "Var", "val": 2}},
{"op": "Or", "F1": {"op": "Var", "val": 2}, "F2": {"op": "Not", "F": {"op": "Or", "F1": {"op": "Var", "val": 1},
"F2": {"op": "Var", "val": 3}}}}]}
"""

In [10]:
# set a similar grn
ref_grn = """
{"object_name": "GRNMain2", "nb_start_gene": 5, "nb_off_trees": 0, "nb_genes": 5,
"params": [[2.007115364074707, 0.3403625786304474, 1.3039988279342651, 2.0225210189819336, 1.5461641550064087],
[0.1, 0.1, 0.1, 0.1, 0.1],
[7.479958534240723, 5.624154090881348, 8.094839096069336, 1.19766366481781, 8.925475120544434],
[9.56630802154541, 3.818467378616333, 9.031471252441406, 7.2058916091918945, 1.283474326133728],
[6.358775615692139, 5.765143394470215, 9.488754272460938, 2.5621650218963623, 5.002293586730957],
[3.9804110527038574, 4.080349922180176, 9.770094871520996, 9.090896606445312, 4.010365009307861],
[5.9285149574279785, 3.148050546646118, 4.948235511779785, 3.5156848430633545, 4.3454718589782715],
[8.12299919128418, 2.998713493347168, 5.796504020690918, 9.65429973602295, 0.45337215065956116]],
"trees": [{"op": "Var", "val": 2}, {"op": "Var", "val": 0},
{"op": "And", "F1": {"op": "Or", "F1": {"op": "Var", "val": 1},
"F2": {"op": "Or", "F1": {"op": "Var", "val": 2}, "F2": {"op": "Var", "val": 3}}},
"F2": {"op": "Not", "F": {"op": "Or", "F1": {"op": "Var", "val": 4}, "F2": {"op": "Var", "val": 0}}}},
{"op": "And", "F1": {"op": "Var", "val": 4}, "F2": {"op": "Var", "val": 2}},
{"op": "Or", "F1": {"op": "Var", "val": 2}, "F2": {"op": "Not", "F": {"op": "Or", "F1": {"op": "Var", "val": 1},
"F2": {"op": "Var", "val": 3}}}}]}
"""

In [11]:
def config_jax2():
    grn = grn_from_json(ref_grn)
    grn.compile()
    ccls = factories["grn2_opti"](grn=grn)

    bb = Brain(time_step=0.5, verbose=False, start_population=4, max_pop_size=1e3,
            cell_cls=ccls, end_time=60, start_time=50, silent=False)
    bb.run()
    return bb

In [12]:
bb = config_jax2()

Ticking abs : 50.0, step : 0.5, size : 16
Ticking abs : 50.5, step : 0.5, size : 10
Ticking abs : 51.0, step : 0.5, size : 10
Ticking abs : 51.5, step : 0.5, size : 12
Ticking abs : 52.0, step : 0.5, size : 17
Ticking abs : 52.5, step : 0.5, size : 20
Ticking abs : 53.0, step : 0.5, size : 22
Ticking abs : 53.5, step : 0.5, size : 34
Ticking abs : 54.0, step : 0.5, size : 39
Ticking abs : 54.5, step : 0.5, size : 49
Ticking abs : 55.0, step : 0.5, size : 70
Ticking abs : 55.5, step : 0.5, size : 83
Ticking abs : 56.0, step : 0.5, size : 120
Ticking abs : 56.5, step : 0.5, size : 146
Ticking abs : 57.0, step : 0.5, size : 192
Ticking abs : 57.5, step : 0.5, size : 255
Ticking abs : 58.0, step : 0.5, size : 332
Ticking abs : 58.5, step : 0.5, size : 430
Ticking abs : 59.0, step : 0.5, size : 574
Ticking abs : 59.5, step : 0.5, size : 745


In [17]:
len(bb.population)

# for segment begining to E60
cells_in_window = [c for c in bb.population.values() if c.appear_time >= 50 and c.appear_time < 60]
print(len(cells_in_window))
cells_divided = [c for c in cells_in_window if c.type().name == "Progenitor"]
print(len(cells_divided))

# filter cell with defined daughter
def all_daughter_defined(cell, population):
    """ Does not check if has daughters ! """
    if not len(cell.children) == 2:
        return False
    
    for daughter_id in cell.children:
        if population[daughter_id].type().name == "Cell":
            return False
    return True

final_cells = list(filter(lambda x: all_daughter_defined(x, bb.population), cells_divided))
print(len(final_cells))

2026
1005
478


In [24]:
tuple(x for x in range(5))

(0, 1, 2, 3, 4)

In [25]:
def pairs(cells, population):
    return [tuple(population[x].type().name for x in cell.children) for cell in cells]

In [28]:
def get_fmetric_pairs(population, min_time=50, max_time=60):
    """ This function is only valid for GRNCells"""
    cells_in_window = [c for c in bb.population.values() 
                       if c.appear_time >= min_time and c.appear_time < max_time]
    cells_divided = [c for c in cells_in_window if c.type().name == "Progenitor"]
    final_cells = list(filter(lambda x: all_daughter_defined(x, bb.population), cells_divided))
    
    return pairs(final_cells, population)

In [29]:
cell_pairs = get_fmetric_pairs(bb.population)

In [30]:
cell_pairs[0]

('Progenitor', 'Progenitor')

In [34]:
def fate_corr(cell_pairs, kC="Progenitor", kN="PostMitotic"):
    counts = Counter(cell_pairs)
    
    CC, CN, NN = counts[(kC, kC)], counts[(kN, kC)] + counts[(kC, kN)], counts[(kN, kN)]
    T = CC + CN + NN
    pCC, pCN, pNN = CC / T, CN / T, NN / T
    all_C, all_N = 2 * CC + CN, 2 * NN + CN
    pC, pN = all_C / (T * 2), all_N / (T * 2)
    eCC, eCN, eNN = pC**2, 2*pC*pN, pN**2
    f_metric = 1 - pCN / eCN
    return f_metric

In [35]:
fate_corr(cell_pairs)

-0.033513513513513615